In [ ]:
baselineResult = pickle.load(open("baseline_cea_T2D_JSON_version_1.1", "rb"))
nest_asyncio.apply()

naiveAccuracies = []
advancedAccuracies = []

naiveF1score = []
advancedF1score = []

for tableResult in baselineResult:
    l, e, f = tableResult
    print(f)
    
    for i in range(len(l)):
        if l[i] == "nichts gefunden in DBlookup":
            l[i] = "a/aasöldkjfnasödj"
    
    allRedirectsSparqlResults = asyncio.run(get_all_wikiPageRedirects(l))
    allRedirectsSparqlResults = np.array(allRedirectsSparqlResults)

    #Sorting the responses because of asynconousity we can not asure the same order
    sortedAllRedirectsSparqlResults = []
    for url in l:
        sparqlResponse = allRedirectsSparqlResults[np.where(allRedirectsSparqlResults == url)[0][0]][0]
        sortedAllRedirectsSparqlResults.append([sparqlResponse, url])
    
    #creating an array with all possible redirectLinks + original url
    #in the end: allpossibleLinksL = [[redirect1, redirect2.., originalUrl], [rdrct1, rdrct2.., oriUrl]...]
    allpossibleLinksL = []
    for qres, url in sortedAllRedirectsSparqlResults:
        linksOfAResource = []
        
        for result in qres['results']['bindings']:
            linksOfAResource.append(result["redirects"]["value"])
            
        allpossibleLinksL.append(linksOfAResource + [url])

    #compute the acccuracy manualy for the advanced method because we have to check
    #wether or not the real url is in the redirect pool instead of naive comparison
    accuracyHelperArray = []
    for i in range(len(l)):
        if e[i] in allpossibleLinksL[i]:
            accuracyHelperArray.append(e[i])
        else:
            accuracyHelperArray.append(0)
    
    advancedAccuracies.append([accuracy_score(e, accuracyHelperArray), f])
    naiveAccuracies.append([accuracy_score(e, l), f])
    
    advancedF1score.append([f1_score(e, accuracyHelperArray, average='micro'), f])
    naiveF1score.append([f1_score(e, l,  average='micro'), f])


#compute the overall accuracy and F1 of both methods
advSumAcc= 0
naiveSumAcc = 0
advSumF1= 0
naiveSumF1 = 0
for i in range(len(advancedAccuracies)):
    advSumAcc += advancedAccuracies[i][0]
    naiveSumAcc += naiveAccuracies[i][0]
    
    advSumF1 += advancedF1score[i][0]
    naiveSumF1 += naiveF1score[i][0]

print("advAcc:", advSumAcc/len(advancedAccuracies))
print("naiveAcc:", naiveSumAcc/len(naiveAccuracies))

print("advf1:", advSumF1/len(advancedF1score))
print("naivef1:", naiveSumF1/len(naiveF1score))

In [ ]:
from itertree import *


root = iTree(tag="root")

def createAllPaths(root, vectorBatches):
    if len(vectorBatches) == 0:
        return root
    else:
        #print(vectorBatches[0])
        for label, vec in vectorBatches[0]:
            #print(label, vec)
            root.append(iTree(tag = label, data=str(vec)))
        copyB = vectorBatches.copy()
        copyB.pop(0)
        #print(len(vectorBatches), len(vectorBatches) > 0)
        for item in root:
            createAllPaths(item, copyB)
    return root

tree = createAllPaths(root, allVectorBatches[0:6])

from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity


cosineSimilarites = []
print(tree.tag)

start = time.time()
for c in root:
    for cc in c:
        for ccc in cc:
            for cccc in ccc:
                for ccccc in cccc:
                    for cccccc in ccccc:
                        vectors = [json.loads(c.d_get()), json.loads(cc.d_get()), json.loads(ccc.d_get()), json.loads(cccc.d_get()), json.loads(ccccc.d_get()), json.loads(cccccc.d_get())]
                        #print(vectors)
                        sim = pairwise_distances(vectors)
                        #print(sim)
                        print(sum(sim[np.triu_indices_from(sim, k = 1)]))
                        #print(pairwise_distances(vectors))
                        cosineSimilarites.append([sum(sim[np.triu_indices_from(sim, k = 1)]) ,c.tag, cc.tag, ccc.tag, cccc.tag, ccccc.tag, cccccc.tag])
print(time.time() -start)

In [5]:
from Levenshtein import distance, ratio
from sklearn.metrics import accuracy_score, f1_score, pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import json
import re 
import html
import aiohttp
import asyncio
import nest_asyncio
import time
import os
import pickle
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, N3
from aiosparql.client import SPARQLClient
from treelib import Node, Tree

In [6]:
def createTree(root, vectorBatches):
    if len(vectorBatches) != 0:
        #print(vectorBatches[0])
        for uri, vec in vectorBatches[0]:
            tree.create_node(tag = uri, data = vec, parent = root.identifier)
        copyB = vectorBatches.copy()
        copyB.pop(0)
        for child in tree.children(root.identifier):
            createTree(child, copyB)
            
def all_paths_to_leaves(self):
        res = []
        
        for leaf in self.leaves():
            vector_path = [self.get_node(nid).data for nid in self.rsearch(leaf.identifier)][-2::-1]
            tag_path = [self.get_node(nid).tag for nid in self.rsearch(leaf.identifier)][-2::-1]
            res.append([vector_path, tag_path])

        return res
    
async def get_kgvec2go_vector(candidate_Uri_pair, session):
    #print(candidate_Uri_pair[0])
    try:
        async with session.get('http://kgvec2go.org/rest/get-vector/dbpedia/'+candidate_Uri_pair[0]) as res:
            data = await res.read()
            try:
                jsonResp = json.loads(data)
                return [candidate_Uri_pair[1], jsonResp["vector"]]
            except:
                return [candidate_Uri_pair[1], [-9999]*200 ]
    except: 
        print("try again")
        async with session.get('http://kgvec2go.org/rest/get-vector/dbpedia/'+candidate_Uri_pair[0]) as res:
            data = await res.read()
            try:
                jsonResp = json.loads(data)
                return [candidate_Uri_pair[1], jsonResp["vector"]]
            except:
                return [candidate_Uri_pair[1], [-9999]*200 ]
            
        
async def get_vectors_for_a_batch(candidateBatch):
    tasks = []
    async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(limit_per_host=20)) as session:
        for candidate_Uri_pair in candidateBatch:
            vector_Uri_pair = asyncio.create_task(get_kgvec2go_vector(candidate_Uri_pair, session))
            tasks.append(vector_Uri_pair)
        all_vector_Uri_pair = await asyncio.gather(*tasks)
    return all_vector_Uri_pair

In [24]:
nest_asyncio.apply()
for number_of_candidates in range(2,3):
    
    baselineResult = pickle.load(open("baseline_cea_T2De_"+str(number_of_candidates)+"_candidates", "rb"))
    
    for number_of_chunks in range(10,12):
        
        output = []
        counter = 1
        
        for tableResult in baselineResult:
            
            labels, baselineUris, trueEntites, allCandidates, filename = tableResult
            counter += 1
            print("handling", filename, counter, number_of_candidates, number_of_chunks)
            
            for index, batch in enumerate(allCandidates):
                if not any(trueEntites[index] in x for y,x in batch):
                    batch.append([trueEntites[index].split("/")[-1], trueEntites[index]])

            #split allCandidates into chunks of n=6 for less intensive similiarity measurments
            allCandidates_chunks = [allCandidates[i:i + number_of_chunks] for i in range(0, len(allCandidates), number_of_chunks)]
            
            #sumC = 0
            #for chunk in allCandidates_chunks:
            #    sumC+= len(chunk)
            #print("länge der chunks zusammen ", sumC)
            
            embeddingsResult = []
            allTableVectors = []
            for candidate_chunk in allCandidates_chunks:
                vectorBatchesTasks = [get_vectors_for_a_batch(candidateBatch) for candidateBatch in candidate_chunk if len(candidateBatch) > 0]
                vector_chunk = await asyncio.gather(*vectorBatchesTasks)
                
                #export all the vectors in the end
                allTableVectors = allTableVectors+vector_chunk

                #create a Tree to compute all the paths
                tree = Tree()
                root = tree.create_node(tag="Root")  # root node
                createTree(root, vector_chunk)

                allPaths = all_paths_to_leaves(tree)

                similiarityMetric = []
                for vecPath, tagPath in allPaths:
                    
                    sim = cosine_similarity(vecPath)
                    #sim = pairwise_distances(vecPath)

                    #to exclude duplicate measurments by pairwise_distance/cosine_sim only take upper triangle from matrix results
                    if len(sim) > 1:
                        simSum = sum(sim[np.triu_indices_from(sim, k = 1)])
                    else:
                        simSum = sum(sim[0])
                    similiarityMetric.append([simSum, tagPath])
                    
                #pick the URIpath that has most similarites between them
                #min or max depending on distance or similarity
                embeddingsResult = embeddingsResult+(max(similiarityMetric, key= lambda k: k[0])[1])
            output.append([labels, embeddingsResult, trueEntites, allTableVectors, filename])
        pickle.dump(output, open( "embeddings_cea_T2De_"+str(number_of_candidates)+"_candidates_"+str(number_of_chunks)+"_chunks_cosineSimilarity_added_trueEntities", "wb"))


handling 1146722_1_7558140036342906956.csv 2 2 10
handling 11688006_0_8123036130090004213.csv 3 2 10
handling 11833461_1_3811022039809817402.csv 4 2 10
handling 12125836_0_1134348206297032434.csv 5 2 10
handling 12193237_0_8699643798888088574.csv 6 2 10
handling 12271141_0_8517913935669973086.csv 7 2 10
handling 13395751_3_369722706637560735.csv 8 2 10
handling 14067031_0_559833072073397908.csv 9 2 10
handling 14380604_4_3329235705746762392.csv 10 2 10
handling 1614988_0_8789868670151796042.csv 11 2 10
handling 16767252_0_2409448375013995751.csv 12 2 10
handling 19073331_0_2742992342272078110.csv 13 2 10
handling 20135078_0_7570343137119682530.csv 14 2 10
handling 21329809_0_5526008408364682899.csv 15 2 10
handling 21362676_0_6854186738074119688.csv 16 2 10
handling 21585935_0_294037497010176843.csv 17 2 10
handling 21801207_0_8144839668470123042.csv 18 2 10
handling 24036779_0_5608105867560183058.csv 19 2 10
handling 24142265_0_4577466141408796359.csv 20 2 10
handling 24417511_0_30583

CancelledError: 